In [0]:
libjpeg_ver = "2.0.2"
virtualGL_ver = "2.6.1"
turboVNC_ver = "2.2.1"

libjpeg_url = "https://svwh.dl.sourceforge.net/project/libjpeg-turbo/{0}/libjpeg-turbo-official_{0}_amd64.deb".format(libjpeg_ver)
virtualGL_url = "https://svwh.dl.sourceforge.net/project/virtualgl/{0}/virtualgl_{0}_amd64.deb".format(virtualGL_ver)
turboVNC_url = "https://svwh.dl.sourceforge.net/project/turbovnc/{0}/turbovnc_{0}_amd64.deb".format(turboVNC_ver)

!curl -fsSL -O $libjpeg_url
!curl -fsSL -O $virtualGL_url
!curl -fsSL -O $turboVNC_url
!dpkg -i *.deb

#install xfce4
#!apt-get update > /dev/null
#!apt-get -qq install xfce4 xfce4-terminal lightdm > /dev/null
!apt-get -qq install xfce4 xfce4-terminal > /dev/null

import pathlib

vnc_sec_conf_p = pathlib.Path("/etc/turbovncserver-security.conf")
vnc_sec_conf_p.write_text("""\
no-remote-connections
no-httpd
no-x11-tcp-connections
""")

# Install TESLA DRIVER FOR LINUX X64 ver410.79.
# Kernel module in this driver is already loaded and cannot be neither removed nor updated.
# (nvidia, nvidia_uvm, nvidia_drm. See dmesg)
# Existing nvidia driver for Xorg is newer than these kernel module and cannot be used with Xorg.
# So overwrite them with the nvidia driver that is same version to loaded kernel module.
!wget -nv -nc http://us.download.nvidia.com/tesla/410.79/NVIDIA-Linux-x86_64-410.79.run
!chmod +x NVIDIA-Linux-x86_64-410.79.run
!./NVIDIA-Linux-x86_64-410.79.run -s --no-kernel-module

#https://virtualgl.org/Documentation/HeadlessNV
!nvidia-xconfig -a --allow-empty-initial-configuration --virtual=1920x1200 --busid PCI:0:4:0

# Edit xorg.conf so that nvidia driver is loaded on seat-1.
import re

with open("/etc/X11/xorg.conf", "r") as f:
  conf = f.read()
  conf = re.sub('(Section "Device".*?)(EndSection)',
                '\\1    MatchSeat      "seat-1"\n\\2',
                conf,
                1,
                re.DOTALL)
#  conf = conf + """
#Section "Files"
#    ModulePath "/usr/lib/xorg/modules"
#    ModulePath "/usr/lib/x86_64-linux-gnu/nvidia-418/xorg/"
#EndSection
#"""

with open("/etc/X11/xorg.conf", "w") as f:
  f.write(conf)

#!service lightdm stop
!/opt/VirtualGL/bin/vglserver_config -config +s +f
#user_name = "colab"
#!usermod -a -G vglusers $user_name
#!service lightdm start

# Run Xorg server
# VirtualGL and OpenGL application require Xorg running with nvidia driver to get Hardware 3D Acceleration.
#
# Without "-seat seat-1" option, Xorg try to open /dev/tty0 but it doesn't exists.
# You can create /dev/tty0 with "mknod /dev/tty0 c 4 0" but you will get permision denied error.
get_ipython().system_raw("Xorg -seat seat-1 -allowMouseOpenFail -novtswitch -nolisten tcp &")

vncrun_py = pathlib.Path("vncrun.py")
vncrun_py.write_text("""\
import subprocess, secrets, pathlib

vnc_passwd = secrets.token_urlsafe()[:8]
vnc_viewonly_passwd = secrets.token_urlsafe()[:8]
print("✂️"*24)
print("VNC password: {}".format(vnc_passwd))
print("VNC view only password: {}".format(vnc_viewonly_passwd))
print("✂️"*24)
vncpasswd_input = "{0}\\n{1}".format(vnc_passwd, vnc_viewonly_passwd)
vnc_user_dir = pathlib.Path.home().joinpath(".vnc")
vnc_user_dir.mkdir(exist_ok=True)
vnc_user_passwd = vnc_user_dir.joinpath("passwd")
with vnc_user_passwd.open('wb') as f:
  subprocess.run(
    ["/opt/TurboVNC/bin/vncpasswd", "-f"],
    stdout=f,
    input=vncpasswd_input,
    universal_newlines=True)
vnc_user_passwd.chmod(0o600)

subprocess.run(
  ["/opt/TurboVNC/bin/vncserver"]
)
""")

!su -c'python3 vncrun.py' colab
#!su -c/opt/TurboVNC/bin/vncserver colab

In [0]:
#Kill TurboVNC
!su -c'/opt/TurboVNC/bin/vncserver -kill :1' colab

In [0]:
!apt-get -o=Dpkg::Use-Pty=0 install mesa-utils

In [0]:
!xdpyinfo -display :0
!/opt/VirtualGL/bin/glxinfo -display :0 -c